In [1]:
# sm = сырая таблица
# sm1 = слайс по нужным столбцам и строкам + столбец с месяцем
# sm2 = анпивот по тарифам в плоскую таблицу

In [11]:
import pandas as pd
from pyxlsb import open_workbook
import io
import csv
from datetime import datetime
import re

In [4]:
filename = r'\\corp.tele2.ru\plm_cluster\All\_Плановый Sales mix\for Tableau\Расчет целевого Sales mix_июнь.xlsb'
print(filename, '...')
with open_workbook(filename) as wb:
    output = io.StringIO()
    writer = csv.writer(output, quoting=csv.QUOTE_NONNUMERIC)
    with wb.get_sheet('Channels_MIX') as sheet:
        for row in sheet.rows():
            csv_line = [r.v for r in row]
            writer.writerow(csv_line)
    output.seek(0)
csvreader = csv.reader(output)

\\corp.tele2.ru\plm_cluster\All\_Плановый Sales mix\for Tableau\Расчет целевого Sales mix_июнь.xlsb ...


In [6]:
output.seek(0)
row1 = next(csvreader)
ordinal_days = int(float(row1[0]))
report_month_dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + ordinal_days - 2).date()
print(f'отчетный месяц: {report_month_dt}')

отчетный месяц: 2020-06-01


In [7]:
col_indexes = [i for i in range(4)] #столбцы
for index, cell_value in enumerate(row1):
    try:
        cell_value = float(cell_value)
        if cell_value in range(161,180): col_indexes.append(index)
    except:
        continue
print(f'столбцы со значениями: {col_indexes}')

столбцы со значениями: [0, 1, 2, 3, 72, 73, 74, 75, 76, 77, 78, 79]


In [8]:
output.seek(0)
for row_number, row in enumerate(csvreader):
    if row[0] == 'Регион': 
        skiprows_count = row_number
        break
else:
    print('Поле "Регион" не найдено')

In [9]:
output.seek(0)
sm = pd.read_csv(output, skiprows = skiprows_count)
sm1 = sm.iloc[:,col_indexes].dropna()
sm1.head(2)

,Регион,Channels,МР,Кластер,Классический план,Мой Tele2 план,Мой разговор план,Мой онлайн план,Мой онлайн + план,Безлимит план,Везде онлайн план,Other план
0,Владимир,Total,Центр,Challenger Elite,1058.0,963.0,2694.0,2886.0,961.0,578.0,0.0,481.0
1,Владимир,MB,Центр,Challenger Elite,276.0,130.0,1187.0,1586.0,684.0,254.0,0.0,395.0


In [22]:
col_names = {
    'Регион':'region',
    'Channels':'channel',
    'МР':'macroregion',
    'Кластер':'cluster',
    'Мой безлимит':'Безлимит',
    'любое старое название':'любое новое название'
}
sm1 = sm1.rename(columns = col_names).rename(columns=lambda x: re.sub(r'(.+) план', r'\1', x))
# sm1.insert(0,'report_month',report_month_dt)
sm1.head(2)

,report_month,region,channel,macroregion,cluster,Классический,Мой Tele2,Мой разговор,Мой онлайн,Мой онлайн +,Безлимит,Везде онлайн,Other
0,2020-06-01,Владимир,Total,Центр,Challenger Elite,1058.0,963.0,2694.0,2886.0,961.0,578.0,0.0,481.0
1,2020-06-01,Владимир,MB,Центр,Challenger Elite,276.0,130.0,1187.0,1586.0,684.0,254.0,0.0,395.0


In [ ]:
fixed_cols_count = 5
sm2 = sm1.melt(id_vars = sm1.columns[:fixed_cols_count].to_list(),
         value_vars=sm1.columns[fixed_cols_count:].to_list(),
         var_name='tariff',
         value_name='subs_count')
sm2 = sm2.loc[(sm2.channel != 'Total'),:].dropna()
sm2.head()

In [ ]:
sm2pivot = sm2.groupby(['tariff','channel']).sum().reset_index()
sm2pivot.pivot(index='tariff', columns='channel', values='subs_count')

In [ ]:
# union_df = sm2
union_df = pd.concat([union_df,sm2]).drop_duplicates()
union_df.shape

In [ ]:
newfilename = r'C:\Users\igor.i.plotnikov\Documents\My Tableau Repository\Datasources\TheGreatDashboard\Sales Mix.xlsx'
union_df.to_excel(newfilename, index = False)

In [ ]:
!jupyter nbconvert --to script "SALES MIX FOR ALL MONTHS.ipynb"